In [1]:
import pandas as pd
import numpy as np

# 导入数据，转时间格式

In [2]:
df = pd.read_csv("./df_saki_timeseries_feature.csv")
df = df.sort_values(["stay_id","time"])
df["time_h"] = df["time"] * 6
df['datetime'] = pd.to_datetime('2022-01-01') + pd.to_timedelta(df['time_h'], unit='h') # 将小时列转换为datetime格式
df = df.drop(["time_h"],axis=1)
df.head(1)

,stay_id,time,calcium,bicarbonate,wbc,spo2,po2,potassium,fio2,dataset,...,lactate,urineoutput,chloride,dbp,glucose,temperature,groupHPD,sodium,baseexcess,datetime
0,5,-2,11.1,21.0,20.600001,98.000002,162.0,4.0,41.0,aumcdb,...,1.0,20.0,105.0,70.2341,76.200002,35.8,2.0,140.0,4.2,2021-12-31 12:00:00


# 提取特征
You can convert each time series into a fixed-dimensional feature vector using libraries like tsfresh（https://tsfresh.readthedocs.io/en/latest/index.html） or catch22（https://github.com/DynamicsAndNeuralSystems/catch22）, and then use the autogluon.tabular.TabularPredictor with problem_type="classification" to classify time series.


## tsfresh b12 - a12
- tsfresh是开源的可以自动提取各种时间特征的python包

In [3]:
# 选择saki前后12h的数据
df = df[df["time"].isin([1,2,3,4])]
df.index = pd.RangeIndex(len(df.index))

fix_feature = [ 'time' , 'dataset' ,'groupHPD'] #,'stay_id', 'datetime'
select_feature = list(set(df.columns.tolist()).difference(fix_feature))

X_timeseries = df[select_feature] 

In [4]:

df_y = df[["stay_id","groupHPD","dataset"]].drop_duplicates()
df_y.index = pd.RangeIndex(len(df_y.index))
y = df_y["groupHPD"]

from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.utilities.distribution import MultiprocessingDistributor
Distributor = MultiprocessingDistributor(n_workers=4,
                                         disable_progressbar=False,
                                         progressbar_title="Feature Extraction")

extracted_features = extract_features(X_timeseries, column_id="stay_id", column_sort="datetime",
                                      impute_function=impute,distributor=Distributor)

extracted_features.index = pd.RangeIndex(len(extracted_features.index))
features_filtered = select_features(extracted_features, y)
features_filtered = pd.concat([df_y,features_filtered], axis=1)
features_filtered.to_csv("./df_saki_tsfresh_generate_features_a1234.csv",index=False)


/home/hanl/miniconda3/envs/sp_shock/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Feature Extraction: 100%|██████████| 20/20 [30:51<00:00, 92.57s/it]  


# 过滤特征

In [5]:
import pandas as pd
import numpy as np

In [6]:
df_fea = pd.read_csv("./df_saki_tsfresh_generate_features_a1234.csv")
X = df_fea.drop(["stay_id","dataset",'groupHPD'],axis=1)
y = df_fea[["groupHPD"]]
X.shape

(8644, 524)

## person相关系数 特征之间的线性相关

In [7]:
import seaborn as sns
corr_df=X.corr()
threshold = 0.9 # 剔除相关性系数高于threshold的corr_drop
upper = corr_df.where(np.triu(np.ones(corr_df.shape), k=1).astype(np.bool)) #返回上三角矩阵
corr_drop = [column for column in upper.columns if any(upper[column].abs() > threshold)]
diff_fea = list(set(X.columns.tolist()).difference(corr_drop))
print("person相关系数大于0.9的特征:",len(corr_drop), "剩余特征数：", len(diff_fea))
X = X[diff_fea]

person相关系数大于0.9的特征: 396 剩余特征数： 128


<ipython-input-7-c0e409dc9687>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_df.where(np.triu(np.ones(corr_df.shape), k=1).astype(np.bool)) #返回上三角矩阵


## 信息量：可以通过计算某个特征对于分类这样的事件到底有多大信息量贡献
- 特征与标签类的互信息越大，特征与标签“相关性越强”。反之。若两者互信息为0,说明两者独立，没什么关系。 


In [8]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.datasets import load_iris
mic_res = mutual_info_classif(X,y,random_state=10)
XT = X.T
XT["MIC"] = mic_res
XT = XT.sort_values("MIC",ascending=False)
XT = XT[XT["MIC"]>0]
XT = XT.drop(["MIC"],axis=1)
XTT = XT.T
df_final = pd.concat([df_fea[["stay_id","dataset",'groupHPD']],XTT],axis=1)

print("互信息筛选后的特征数量：",XTT.shape[1])

/home/hanl/miniconda3/envs/sp_shock/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


互信息筛选后的特征数量： 115


In [9]:
df_final.to_csv("./df_saki_tsfresh_generate_features_a1234_CorrMICfilt.csv",index=False)